In [1]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#Lets define the URL

url="https://www.worldometers.info/coronavirus/"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

In [3]:
#we pick the id of the table we want to scrape and extract HTML code for that particular table only
covid_table = soup.find("table", attrs={"id": "main_table_countries_today"})

In [4]:
#the head will form our columns
head = covid_table.thead.find_all("tr") 
head #the headers are contained in this HTML code

[<tr>
 <th width="1%">#</th>
 <th width="100">Country,<br/>Other</th>
 <th width="20">Total<br/>Cases</th>
 <th width="30">New<br/>Cases</th>
 <th width="30">Total<br/>Deaths</th>
 <th width="30">New<br/>Deaths</th>
 <th width="30">Total<br/>Recovered</th>
 <th width="30">New<br/>Recovered</th>
 <th width="30">Active<br/>Cases</th>
 <th width="30">Serious,<br/>Critical</th>
 <th width="30">Tot Cases/<br/>1M pop</th>
 <th width="30">Deaths/<br/>1M pop</th>
 <th width="30">Total<br/>Tests</th>
 <th width="30">Tests/<br/>
 <nobr>1M pop</nobr>
 </th>
 <th width="30">Population</th>
 <th style="display:none" width="30">Continent</th>
 <th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
 <th width="30">New Cases/1M pop</th>
 <th width="30">New Deaths/1M pop</th>
 <th width="30">Active Cases/1M pop</th>
 </tr>]

In [5]:
headings = []
for th in head[0].find_all("th"):
    # remove any newlines and extra spaces from left and right
    print(th.text)
    #headings.append(td.b.text.replace('\n', ' ').strip())
    headings.append(th.text.replace("\n","").strip())
print(headings)

#
Country,Other
TotalCases
NewCases
TotalDeaths
NewDeaths
TotalRecovered
NewRecovered
ActiveCases
Serious,Critical
Tot Cases/1M pop
Deaths/1M pop
TotalTests
Tests/
1M pop

Population
Continent
1 Caseevery X ppl
1 Deathevery X ppl
1 Testevery X ppl
New Cases/1M pop
New Deaths/1M pop
Active Cases/1M pop
['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl', 'New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop']


In [6]:
body = covid_table.tbody.find_all("tr") 
body[0] #here is one example of HTML snippet for one row

<tr class="total_row_world row_continent" data-continent="North America" style="display: none">
<td></td>
<td style="text-align:left;">
<nobr>North America</nobr>
</td>
<td>127,000,199</td>
<td></td>
<td>1,637,356</td>
<td></td>
<td>122,980,768</td>
<td>+626</td>
<td>2,382,075</td>
<td>5,948</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td data-continent="North America" style="display:none;">North America</td>
<!-- 1 Case every X -->
<td>
</td>
<!-- 1 Death every X -->
<td></td>
<!-- 1 test every X -->
<td></td>
<td></td>
<td></td>
<td></td>
</tr>

In [7]:
#lets declare empty list data that will hold all rows data
data = []
for r in range(1,len(body)):
    row = [] # empty lsit to hold one row data
    for tr in body[r].find_all("td"):
        row.append(tr.text.replace("\n","").strip())
        #append row data to row after removing newlines escape and triming unnecesary spaces
    data.append(row)

In [8]:
#We can now pass data into a pandas dataframe
#with headings as the columns
df = pd.DataFrame(data,columns=headings)
df.head(10)


,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,Asia,"218,134,419",+334,"1,547,613",,"201,916,516","+16,996","14,670,290","15,101",...,,,,Asia,,,,,,
1,,Europe,"249,665,850","+4,728","2,065,787",+91,"245,795,069","+9,281","1,804,994","5,531",...,,,,Europe,,,,,,
2,,South America,"68,820,634",,"1,357,470",,"66,485,092",,"978,072","10,097",...,,,,South America,,,,,,
3,,Oceania,"14,538,582",,"29,206",,"14,394,010",,"115,366",69,...,,,,Australia/Oceania,,,,,,
4,,Africa,"12,830,605",,"258,804",,"12,087,518",,"484,283",547,...,,,,Africa,,,,,,
5,,,721,,15,,706,,0,0,...,,,,,,,,,,
6,,World,"690,991,010","+5,062","6,896,251",+91,"663,659,679","+26,903","20,435,080","37,293",...,,,,All,,,,,,
7,1,USA,"107,329,712",,"1,168,267",,"105,479,716",,"681,729",782,...,"1,180,815,103","3,526,871","334,805,269",North America,3,287,0,,,"2,036"
8,2,India,"44,994,351",,"531,908",,"44,460,975",,"1,468",N/A,...,"930,797,975","661,721","1,406,631,776",Asia,31,"2,645",2,,,1
9,3,France,"40,138,560",,"167,642",,"39,948,906","+2,631","22,012",869,...,"271,490,188","4,139,547","65,584,518",Europe,2,391,0,,,336


In [9]:
data = df[df["#"]!=""].reset_index(drop=True)

data = data.drop_duplicates(subset = ["Country,Other"])


In [10]:
data.columns


Index(['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths',
       'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases',
       'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests',
       'Tests/1M pop', 'Population', 'Continent', '1 Caseevery X ppl',
       '1 Deathevery X ppl', '1 Testevery X ppl', 'New Cases/1M pop',
       'New Deaths/1M pop', 'Active Cases/1M pop'],
      dtype='object')

In [11]:
# Columns to keep
cols = ['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths',
       'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases',
       'Serious,Critical', 'TotalTests',"Continent"]

In [12]:
data_final = data[cols]
data_final.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalTests,Continent
0,USA,"107,329,712",,"1,168,267",,"105,479,716",,"681,729",782,"1,180,815,103",North America
1,India,"44,994,351",,"531,908",,"44,460,975",,"1,468",N/A,"930,797,975",Asia
2,France,"40,138,560",,"167,642",,"39,948,906","+2,631","22,012",869,"271,490,188",Europe
3,Germany,"38,428,685",,"174,352",,"38,240,600",,"13,733",N/A,"122,332,384",Europe
4,Brazil,"37,671,420",,"703,964",,"36,249,161",,"718,295",N/A,"63,776,166",South America
